In [1]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
from pycocotools.coco import COCO

In [37]:
data = pd.read_json("./full_train.json")
confidence = 0.001
dataset = "train" 

image_id                                        0

category_id                                     7

bbox           [607.478, 518.568, 346.8, 499.062]

score                                     0.97135

test/0015.jpg

In [38]:
prediction_strings = []
file_names = []
for i in tqdm(data.groupby(by=["image_id"])):
    image_id = i[0]
    prediction_string = ""
    for j in i[1].iterrows():
        if j[1].score < confidence:
            continue
        blk = [j[1].category_id, round(j[1].score,15), j[1].bbox[0], j[1].bbox[1], j[1].bbox[0] + j[1].bbox[2], j[1].bbox[1] + j[1].bbox[3]]
        blk = list(map(lambda x : str(x) + " " ,blk))
        for i in blk:
            prediction_string += i
        #print(blk)
        #print(j[1]) 
    file_name = dataset + "/" + str(image_id + 10000)[1:] + ".jpg"

    prediction_strings.append(prediction_string)
    file_names.append(file_name)


100%|██████████| 3906/3906 [00:37<00:00, 102.83it/s]


In [39]:

submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('./submission.csv', index=None)
submission.head()

,PredictionString,image_id
0,0 0.97872 199.491 193.15 745.552 662.52 7 0.00...,test/0000.jpg
1,3 0.97974 265.743 162.129 897.924 675.842 7 0....,test/0002.jpg
2,2 0.97316 462.929 369.406 695.233 627.20800000...,test/0003.jpg
3,1 0.96775 858.852 412.719 1023.088 613.342 1 0...,test/0004.jpg
4,1 0.98052 494.124 626.738 842.008 949.46600000...,test/0005.jpg


---
## Confidence edit  
### COCO json-> CSV
에러 수정 중

In [35]:
coco = COCO("./csv_to_json2.json")
confidence = 0.4

loading annotations into memory...


AssertionError: annotation file format <class 'list'> not supported

In [28]:
file_names = []
strings = []
for image_id in tqdm(coco.getImgIds()):
    image_info = coco.loadImgs(image_id)[0]
    annotation_id = coco.getAnnIds(imgIds=image_info['id'])
    annotation_info_list = coco.loadAnns(annotation_id)
    string = ""
    for ano in annotation_info_list:
        if ano['score'] < confidence:
            continue
        blk = [ano['category_id'], ano['score'], ano['bbox'][0], ano['bbox'][1], ano['bbox'][2] + ano['bbox'][0], ano['bbox'][3] + ano['bbox'][1]]
        blk = list(map(lambda x : str(x) + " " ,blk))
        for i in blk:
            string += i
    
    file_name = "test/" + image_info['file_name']
    strings.append(string)
    
    file_names.append(file_name)

100%|██████████| 4871/4871 [00:00<00:00, 9063.42it/s]


In [29]:
submission = pd.DataFrame()
submission['PredictionString'] = strings
submission['image_id'] = file_names
submission.to_csv('./submissionCOCO.csv', index=None)
submission.head()

,PredictionString,image_id
0,7 0.98287 604.825 514.406 956.942 1023.5919999...,test/test/0000.jpg
1,5 0.98095 345.005 251.478 754.806 696.483 5 0....,test/test/0001.jpg
2,1 0.97 301.633 310.344 1019.517 748.543 1 0.96...,test/test/0002.jpg
3,9 0.97585 139.554 264.641 914.52 830.3 9 0.949...,test/test/0003.jpg
4,1 0.9523 187.535 253.625 872.747 774.303 7 0.9...,test/test/0004.jpg
